In [1]:
import os
import sys
#list all folders in /scratch/groups/rondror/marvinli/combind_fragment/benchmark_set_split_no_copy
import glob

features_screen_path = '/scratch/groups/rondror/marvinli/combind_fragment/benchmark_set_split_no_copy/*/features_screen/*'
features_screen_path = glob.glob(features_screen_path)
sys.path.append('/scratch/groups/rondror/marvinli/combind_fragment/')


In [2]:
from combind_find import screen

: 

In [8]:
screen

NameError: name 'screen' is not defined